In [28]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

plt.rcParams['figure.figsize'] = (10,10)
plt.rcParams.update({'font.size': 12})

In [32]:
def getImageComponents(fp):
    img = cv2.imread(fp, cv2.IMREAD_GRAYSCALE)
    nrows = img.shape[0]
    height = int(np.ceil(nrows/3))
    g_start = int(np.round((nrows - height)/2))
    
    #b,g,r = np.split(img, 3)
    b = img[:height, :]#,:]
    g = img[g_start:g_start+height, :]#,:]
    r = img[-height:, :]#,:]
    return b,g,r

DIRECTORY = 'DataSamples'
fp = os.path.join(DIRECTORY, 's1.jpg')
b,g,r = getImageComponents(fp)

# perform image alignment
res = cv2.matchTemplate(b,g,cv2.TM_CCOEFF_NORMED)
loc = np.where(res >= 0.8)
loc
#https://www.pyimagesearch.com/2014/09/15/python-compare-two-images/

#color_img = cv2.merge((b,g,r))
#plt.imshow(color_img)

(array([0], dtype=int64), array([0], dtype=int64))

In [9]:
a = np.array([[[1],[2],[3]],[[4],[5],[6]],[[7],[8],[9]]])
b,c,d = np.split(a,3)
b

array([[[1],
        [2],
        [3]]])

In [15]:
a = [1,2,3,4,5]
a[-2:]
np.round(34.4)

34.0